# Tools to simulate an audit

Create voted ballots, election results, ballot manifest, and audit instructions.


In [1]:
import math
from collections import OrderedDict
import numpy as np

In [2]:
ballot_fn = './ballots.md'
manifest_fn = './manifest.md'
worksheet_fn = './worksheet.md'
instruct_fn = './instructions.md'

tail = '\n \\newpage\n'
checkboxes = ['- [ ] ', '- [x] ']

n_ballots = 100
n_precincts = 5
ballots_per_precinct = [25, 15, 20, 25, 15] 
vote_dict = OrderedDict({'Alice': 63, 'Bob': 27, 'Carol': 6})
kelly_lam = (vote_dict['Alice']- vote_dict['Bob'])/(vote_dict['Alice'] + vote_dict['Bob']) # Kelly bet for AvB.
print(f'{kelly_lam=}')

kelly_lam=0.4


In [3]:
manifest = '## Reported results \n| Candidate | votes |\n |:-----|-----:|\n'
for c, v in vote_dict.items():
    manifest += f'| {c} | {v} |\n'
manifest += f'| invalid votes | {n_ballots-np.sum(np.fromiter((v for v in vote_dict.values()), dtype=int))} |\n\n'
manifest += '## Ballot manifest\n| Precinct | cards | serial no. range | \n |:-----|-----:|:-----:|\n'
first = 1
for i, b in enumerate(ballots_per_precinct):
    manifest += f'| {str(i+1)} | {str(b)} | {first} &ndash; {first+b-1} \n'
    first += b

In [4]:
with open(manifest_fn, 'w') as manif:
    manif.write(manifest)

In [5]:
votes = np.array([])
n_votes = 0
for c, v in vote_dict.items():
    n_votes += v
    votes = np.concatenate((votes, np.repeat(c, v)))
votes = np.concatenate((votes, np.repeat('', n_ballots-n_votes)))
votes = np.random.permutation(votes)

In [6]:
ballots = ''
v = 0
for precinct in range(n_precincts):
    for b in range(ballots_per_precinct[precinct]):
        ballots += f'**Precinct {str(precinct+1)}**\n\n## Mayor of Voting Village \n\n### (Vote for one)\n\n'
        for c in vote_dict.keys():
            ballots += (checkboxes[1] if c==votes[v] else checkboxes[0]) + c + ' \n'
        ballots += tail
        v += 1

In [7]:
with open(ballot_fn, 'w') as out:
    out.write(ballots)

In [8]:
tab = '''\\pagenumbering{gobble}
## Audit worksheet
\\begin{tabular}{r|r|c|r|c|r|r|r}\\hline
     &          &          &                     &       & \\multicolumn{3}{c}{Running totals} \\cr
     \\cline{6-8}
draw & roll + 1 & precinct & ballot w/i precinct & vote  & Alice/Bob  & Alice/Carol & Bob/Alice  \\cr
\\hline
  1   &          &          &                     &       &     0      &      0      &   0        \\cr'''
for i in range(49):
    tab += f'\\hline {i+2} & & & & & & & \\cr\n'

tab += '\\end{tabular}\\newpage'
tab += '''\\begin{tabular}{r|r|c|r|c|r|r|r}\\hline
     &          &          &                     &       & \\multicolumn{3}{c}{Running totals} \\cr
     \\cline{6-8}
draw & roll + 1 & precinct & ballot w/i precinct & vote  & Alice v Bob  & Alice v Carol & Bob v Alice  \\cr
\\hline
  51   &          &          &                     &       &     0      &      0      &   0        \\cr'''
for i in range(49):
    tab += f'\\hline {i+52} & & & & & & & \\cr\n'

tab += '\\end{tabular}'

with open(worksheet_fn, 'w') as worksheet:
    worksheet.write(tab)

## Calculations for the RLA


In [9]:
# search for optimal bet; this should "discover" the Kelly bet for the closest pairwise margin

alpha = 0.1  # risk limit
reps = 50000 # replications to simulate sample size
sizes = []
delta = 0.01
q = [50, 90]
for lam in np.arange(kelly_lam-5*delta, kelly_lam+6*delta, delta):
# lam = 0.38939067  # fraction of your fortune that you win or lose
# lam = 0.22035
# lam = 0.25791
    c = math.log(1+lam)          # this will be the unit "up" step
    lose = -math.log(1-lam)/c    # express the "down" step as a multiple of c
    thresh = math.log(1/alpha)/c # express the termination threshold as a multiple of c
    ssize = 0
    ssizes = []
    for r in range(reps):
        n = 0
        t_ab = 0
        t_ac = 0
        while (t_ab < thresh or t_ac < thresh):
            n += 1
            v = np.random.choice(votes)
            match v:
                case 'Alice':
                    t_ab += 1
                    t_ac += 1
                case 'Bob':
                    t_ab -= lose
                case 'Carol':
                    t_ac -= lose
        ssize += n
        ssizes.append(n)
    expect = ssize/reps
    qtiles = np.percentile(ssizes,q)
    if np.isclose(lam, kelly_lam):
        expect_size = expect
    print(f'{lam=} {c=} {lose=} {thresh=} {expect=} {qtiles=}')
    sizes.append([lam, c, lose, thresh, expect, qtiles])

lam=np.float64(0.35000000000000003) c=0.30010459245033816 lose=1.4354425987790944 thresh=7.672608653514963 expect=33.7739 qtiles=array([26., 65.])
lam=np.float64(0.36000000000000004) c=0.3074846997479607 lose=1.4514123889553938 thresh=7.488454205628542 expect=33.68694 qtiles=array([26., 66.])
lam=np.float64(0.37000000000000005) c=0.3148107398400336 lose=1.4676610455899164 thresh=7.3141884999351365 expect=33.43322 qtiles=array([26., 65.])
lam=np.float64(0.38000000000000006) c=0.3220834991691134 lose=1.4841983590472674 thresh=7.149031536648355 expect=33.79322 qtiles=array([26., 66.])
lam=np.float64(0.39000000000000007) c=0.32930374714260047 lose=1.5010346104586902 thresh=6.992283303709092 expect=33.11364 qtiles=array([24., 67.])
lam=np.float64(0.4000000000000001) c=0.336472236621213 lose=1.5181806050198965 thresh=6.843313778623002 expect=33.1012 qtiles=array([24., 67.])
lam=np.float64(0.4100000000000001) c=0.343589704390077 lose=1.5356477081261763 thresh=6.701554393434105 expect=33.51338

In [10]:
# expected sample sizes
lam = kelly_lam            # optiimal for sampling with replacement for the contest with smallest margin
c = math.log(1+lam)        # "up" step
lose = -math.log(1-lam)/c  # "down" step as a multiple of c
thresh_10 = math.log(10)/c # threshold for 10% risk limit as a multiple of c
thresh_5 = math.log(20)/c  # threshold for 5% risk limit as a multiple of c
print(f'{lam=} {c=} {lose=} {thresh=}')

# clean up the numbers for exposition
lose_rnd = round(lose,1)
thresh_10_rnd = math.ceil(thresh_10)
thresh_5_rnd = round(thresh_5,1)

lam=0.4 c=0.3364722366212129 lose=1.5181806050198963 thresh=6.197015431497111


## Audit instructions

In [11]:
instructions = f'''
# Audit demo

## Philip B. Stark, pbstark@berkeley.edu
### License: CC-BY-NC-ND

## Materials 

+ voted ballot cards
    - there are 100 ballot cards
    - each card is labeled with a precinct (top of page) and a serial number (bottom of page)
    - each card should have a vote for at most one candidate
    - if a card does not show a vote for any candidate, it is an _undervote_: it does not count as a vote for any candidate
    - if a card has a mark for more than one candidate, it is an _overvote_: it does not count as a vote for any candidate
+ reported results: the number of valid votes each candidate was reported to have received. 
+ ballot manifest
    - explains how the ballot cards are stored:
    - for each precinct, lists the number of ballot cards and the serial number range of those cards
+ two 10-sided dice of different colors
+ audit log and worksheet (below)
+ a pen

## Instructions

Alice was the reported winner of the contest. 
The goal of the audit is to check whether Alice really won.
If she did, the audit will usually stop without looking at every ballot card. 
If Alice did _not_ really win, the audit has a large chance of leading to a full hand count to determine who did win.

The audit examines randomly selected ballot cards; it does not use information from the electronic voting system.
This type of audit is called "ballot polling."
(There are RLA methods that use other information from the voting system, for instance, vote subtotals for precincts
or the machines' record of the votes on individual ballot cards.)
Ballot polling has similarities to an exit poll, except that instead of asking people how they voted, 
it looks at ballots to see what votes were recorded.
Unlike a survey of voters, the ballots have to answer, and they have to answer truthfully.
(The goal is also different: a risk-limiting audit does not try to estimate vote shares,
only to determine who won.)

RLAs can be based on sampling _with replacement_ (the same card can be selected
more than once) or on sampling _without replacement_ (each card is selected at most once).
The formulas for computing the risk using sampling without replacement are more complicated.
This demo uses sampling _with_ replacement so that the calculations can be done by hand.
If the same card is selected more than once, there is no need to _retrieve_ it more than once: 
you already know what vote it shows.

For this demo, the expected number of cards you need to inspect to confirm the outcome is about {int(expect_size)}, 
including duplicates. 
Because the sample is drawn with replacement, the audit workload depends on the candidates _shares_
of the vote, not on the total number of votes or cards cast.
That is, the number of draws required to confirm the outcome would be expected to be 
about {int(expect_size)} cards whether there were 100 ballot cards
or 1,000,000 ballot cards in the election.
The only difference is that when the election is smaller, it's more likely that the sample will contain
the same card more than once, so the number of cards that have to be _retrieved_ is expected to be smaller.
Of course, {int(expect_size)} is a much larger fraction of 100 than it is of 1,000,000.

The audit is conducted using two running totals, both of which start at zero.
One running total summarizes the evidence that Alice got more votes than Bob. 
Bigger values are stronger evidence that Alice got more votes than Bob.
Negative values are evidence that Bob got more votes than Alice--that the reported outcome is wrong!
The other running total summarizes the evidence that Alice got more votes than Carol. 
Bigger values are stronger evidence that Alice got more votes than Carol.


1. Divide the ballots into five piles according to the precinct printed at the top. 
2. Decide which of the two dice will represent the "tens" digit and which will represent the "ones" digit.
3. Select a ballot card at random by rolling the dice to generate a two-digit random number between 00 and 99, then adding 1 to get a random number between 1 and 100 (the number of ballots).
4. Write that number in column 2 of the worksheet.
5. Retrieve the ballot with that serial number. The _ballot manifest_ will help you find it: it shows the range of
serial numbers each precinct contains.
6. Write the precinct in column 3 of the worksheet, and the position of the ballot within the precinct in column 4 of the worksheet. 
7. Retrieve a randomly selected card and read the vote from it.
8. Write the vote in column 5 of the worksheet.
    - if it is an undervote or overvote, don't change any running total; return to step 3.
    - if it is a vote for Alice, add 1 to the running totals in columns 6 and 7
    - if it is a vote for Bob, subtract {lose_rnd} from the running total for Alice v Bob (column 6) but don't change the running total for Alice v Carol (column 7)
    - if it shows a vote for Carol, subtract {lose_rnd} from the running total for Alice v Carol (column 7), but don't change the running total in column for Alice v Bob (column 6)
    - ignore column 8 for now
9. Repeat steps 3--8 until either:
    - the running totals in columns 6 and 7 hit or exceed {thresh_10_rnd} (if one of the running totals hits {thresh_10_rnd} or greater, you can stop updating it: the audit has confirmed the corresponding comparison)
    - you get bored or discouraged and decide it's easier to do a full hand count of the votes on all the cards (which will show who really won)
 
This is a risk-limiting audit with a risk limit of 10%.
If you use a stopping threshold of {thresh_5_rnd} instead of {thresh_10_rnd}, it is a risk-liimiting audit with a risk limit of 5%.

## What if Alice had really lost?

Suppose that Bob had been the reported winner instead of Alice.
We will use column 8 of the worksheet to see what the audit would have done, using the data in columns 1-5.
In each row, update column 8 as follows: 

+ if the vote in column 5 is a vote for Bob, add 1 to the running total in column 8
+ if the vote in column 5 is a vote for Alice, subtract {lose_rnd} from the running total in column 8
+ otherwise, don't change the running total in column 8

You should see that the running total tends to _decrease_ rather than _increase_, since Alice really won:
the evidence that Bob beat Alice tends to get weaker and weaker.
The chance that the running total in column 8 ever reaches the threshold 7 is at most 10%, the _risk limit_ of the audit.

## Other types of elections

The same core ideas can be used to audit almost every social choice function used in political elections, including
multi-winner plurality, supermajority, instant-runoff voting (IRV) and ranked-choice voting, Borda count,
all "scoring rules," D'Hondt, and Hamiltonian.
See Stark, P.B., 2020. https://arxiv.org/abs/1911.10035
For single transferrable vote, a form of multi-winner ranked-choice voting, RLA methods are known only for special cases.

## Where does the stopping rule come from?

The chance that a randomly selected card shows a vote for Alice is equal to the fraction of cards in the election
that have votes for Alice.
For instance, if there are {n_ballots} cards in all, of which 
{vote_dict['Alice']} have a vote for Alice, then the chance that a 
randomly selected card has a vote for Alice is {vote_dict['Alice']}/{n_ballots} = 
{np.round(100*vote_dict['Alice']/n_ballots, decimals=1)}%.

The _conditional_ chance that a randomly selected card shows a vote for Alice, given that it shows a vote for Alice or Bob,
is the fraction of cards that show a vote for Alice among cards that show a vote for Alice or a vote for Bob.
So, for instance, if {vote_dict['Alice']} cards have a vote for Alice and {vote_dict['Bob']}
have a vote for Bob, the conditional chance that
a randomly selected card shows a vote for Alice given that it shows a vote for Alice or for Bob is 
{vote_dict['Alice']}/({vote_dict['Alice']}+{vote_dict['Bob']}) = 
{np.round(100*vote_dict['Alice']/(vote_dict['Alice']+ vote_dict['Bob']), decimals=2)}%.

Alice really beat Bob if she got more votes than Bob, i.e., if the conditional chance that a randomly 
selected card shows a vote for Alice given that it shows a vote for Alice or for Bob is greater than 50%.

Similarly, the _conditional_ chance that a randomly selected card shows a vote for Alice, given that it 
shows a vote for Alice or Carol,
is the fraction of cards that show a vote for Alice among cards that show a vote for Alice or a vote for Carol.
So, for instance, if {vote_dict['Alice']} cards have a vote for Alice and 
{vote_dict['Carol']} have a vote for Carol, the conditional chance that
a randomly selected card shows a vote for Alice given that it shows a vote for Alice or for Carol is 
{vote_dict['Alice']}/({vote_dict['Alice']}+{vote_dict['Carol']}) = 
{np.round(100*vote_dict['Alice']/(vote_dict['Alice']+vote_dict['Carol']), decimals=2)}%.

Alice really beat Carol if she got more votes than Carol, i.e., if the conditional chance that a randomly 
selected card shows a vote for Alice given that it shows a vote for Alice or for Carol is greater than 50%.

### Fair bets

A bet on a random event is _fair_ if you expect to break even.
Here, _expect_ has a mathematical meaning: it's the amount you get if you win times the chance you win, plus the amount you get if you lose times the chance you lose.

For instance, if you bet \\$1 that the toss of a fair coin will land "heads," 
you expect to break even: the chance you win \\$1 is 50% and the chance you lose \$1 is 50%.
If the chance of heads is less than 50%, the bet is _unfavorable_; if it's greater than 50%, the bet is _favorable_.

Similarly, if you bet \\$1 and get back \\$1.50 if the coin lands heads and \\$0.50 if the coin lands tails,
you also expect to break even: the chance you get \\$1.50 is 50% and the chance you get \\$0.50 is 50%,
and the expected payoff is \\$1.50 $\\times$ 50% + \\$0.50 $\\times$ 50% = \\$1, the amount of your stake.
If the chance of heads is less than 50%, the bet is _unfavorable_; if it's greater than 50%, the bet is _favorable_.

A bet is _unfavorable_ if you expect to lose money. 
For instance, if you bet \\$1 on the toss of a fair coin and get paid \\$1.50 if it lands heads and \\$0 if it lands tails, you expect to lose \\$0.25 on average each time you play.
Most casino games are _unfavorable_.

A French mathematician named Jean Ville proved in 1939 that in any sequence of fair or unfavorable bets, 
the chance you ever multiply your initial bankroll by any number $c$ is at most $1/c$, if you're not allowed to borrow money (that is, if you go broke, you're out).
For example, if you start with a bankroll of \\$1, the chance your fortune ever reaches \\$10 is at most $1/10$, i.e., 10%.
The chance your fortune ever reaches \\$20 is at most $1/20$, i.e., 5%.

The audit method we are using works because it amounts to placing two series of bets,
one involving Alice v. Bob and the other involving Alice v. Carol.
We play both of these games simultaneously, starting with a stake of \\$1 in each.
We aren't allowed to place a bet we can't cover, and if we ever go broke, we're out of the game (and the audit
becomes a full hand count).
The series of games for Alice v. Bob are fair or subfair unless Alice got more votes than Bob;
the series of games for Alice v. Carol are fair or subfair unless Alice got more votes than Carol.

To check whether Alice got more votes than Bob, we repeatedly bet that the next card drawn shows a vote for Alice given
that it shows a vote for Alice or Bob:

+ if it shows a vote for Alice, we win
+ if it shows a vote for Bob, we lose
+ if it doesn't show a vote for Alice or for Bob, no money changes hands
  
To check whether Alice got more votes than Carol, we repeatedly bet that the next card drawn shows a vote for Alice given
that it shows a vote for Alice or Carol:

+ if it shows a vote for Alice, we win
+ if it shows a vote for Carol, we lose
+ if it doesn't show a vote for Alice or for Carol, no money changes hands

We need to set the payoff so that the games are fair or unfavorable unless Alice got more votes than the
other candidate.

+ If we manage to multiply our initial stake by 10 in the Alice v Bob game, then either Alice beat Bob,
or something happened that should happen at most 10% of the time.
+ If we manage to multiply our initial stake by 10 in the Alice v Carol game, then either Alice beat Carol,
or something happened that should happen at most 10% of the time.
+ If we manage to multiply our initial stake in **both** games by 10, then either Alice really beat _both_ Bob
and Carol, and thus really won the election, or something happened that should happen at most 10% of the time.
That amounts to an RLA with a risk limit of 10%.

Similarly, 
If we manage to multiply our initial stake in both games by 20 in both games, then either Alice really won
the election, or something happened that should happen at most 5% ot the time.
If we agree that if we don't multiply our stake by 20 (e.g., if we go broke or get bored) we will perform
an accurate, full hand count, the resulting procedure is an RLA with a risk limit of 5% (1/20).

The numbers in this demo correspond to betting the same fraction of your bankroll every time.
You get back your stake plus {np.round(100*lam, decimals=1)}% if you win
the bet, and you get back your stake minus {np.round(100*lam, decimals=1)}% if you lose the bet.^[The fraction {np.round(100*lam, decimals=1)}%
is optimal--grows your wealth fastest--when Alice really has {np.round(100*vote_dict['Alice']/(vote_dict['Alice']+ vote_dict['Bob']), decimals=2)}%
of the valid votes for the pair being compared, as she does in the comparison with Bob.
We could bet different percentages of our bankroll in the two games.
If we did, the optimal bet for checking whether Alice got more votes than Carol would be
{np.round(100*(vote_dict['Alice']- vote_dict['Carol'])/(vote_dict['Alice'] + vote_dict['Carol']), decimals=2)}% of your bankroll
instead of {np.round(100*vote_dict['Alice']/(vote_dict['Alice']+ vote_dict['Bob']), decimals=2)}%.
The general rule for sampling with replacement (proved by John Kelly Jr. in 1956) is that to grow your wealth fastest, 
bet the fraction (votes for reported winner - votes for reported loser)/(votes for reported winner + votes for reported loser)
of your bankroll on each draw.]
Instead of _multiplying_ your stake by {np.round(1+lam, decimals=3)} if you win or by 
(1-{np.round(lam, decimals=3)}) if you lose,
the demo uses logarithms, so that multiplication becomes addition.
The number {np.round(100*lam, decimals=1)}% (more precisely, {100*lam}%) was chosen to minimize the expected sample
size and to make the 
arithmetic easier to do by hand:
what you subtract from the running total if you lose is quite close to {lose_rnd} times what you add if you win
(a more precise value is {lose}).
The threshold for stopping, {thresh_10_rnd}, has also been rounded up for simplicity: a more precise value is {thresh_10}.
'''

with open(instruct_fn, 'w') as instr:
    instr.write(instructions)

In [12]:
!pandoc ballots.md -o ballots.pdf; pandoc manifest.md -o manifest.pdf; pandoc worksheet.md -o worksheet.pdf; pandoc instructions.md -o instructions.pdf